In [ ]:
import gc
from ultralytics import YOLO
import torch
import os
import shutil
import gc

def train_and_test_yolo_face():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

    # --- 1. SETUP MODEL ---
    model_name = 'yolo26n.pt' 
    print(f"🔄 Loading model base: {model_name}...")
    model = YOLO(model_name) 

    # --- 2. TRAINING  ---
    print("🚀 Mulai Training YOLO26 Face ...")
    
    results = model.train(
        data='data.yaml',
        project='YOLO26_Face',  
        name='YOLO26n_Face_Run',  
        
        epochs=100,
        patience=0,              
        imgsz=640,               
        batch=-1,                
        workers=8,               
        device=0,
        cache=True,              
        amp=True,                
        rect=True,               
        
        optimizer='auto',        
        cos_lr=True,             
        close_mosaic=10,         
        single_cls=True,         
        
        mosaic=1.0,              
        degrees=10.0,            
        fliplr=0.5,              
        
    )

    # --- 3. SAVE BEST MODEL ---
    print("\n💾 Menyimpan model terbaik...")
    
    best_weight_path = os.path.join(results.save_dir, 'weights', 'best.pt')
    target_name = 'yolo26n-face.pt'
    
    if os.path.exists(best_weight_path):
        shutil.copy(best_weight_path, target_name)
        print(f"✅ Model tersimpan: {target_name}")
        print(f"📂 Lokasi asli: {results.save_dir}")
    else:
        print("❌ Warning: File best.pt tidak ditemukan.")
        return

    # --- 4. EXPORT TO ONNX ---
    print("\n📦 Exporting ke ONNX (Dynamic)...")
    final_model = YOLO(target_name)
    final_model.export(format='onnx', dynamic=True, simplify=True)

    # --- 5. BENCHMARKING ---
    print("\n📊 Melakukan Benchmark Akhir...")
    metrics = final_model.val(
        data='widerface.yaml',
        split='test',    
        imgsz=640,
        batch=16,
        conf=0.25,
        iou=0.6
    )
    
    print(f"\n🏆 HASIL AKHIR ({target_name}):")
    print(f"   🎯 mAP@50    : {metrics.box.map50:.2%}")
    print(f"   🎯 mAP@50-95 : {metrics.box.map:.2%}")
    try:
        print(f"   ⚡ Speed     : {metrics.speed['inference']:.2f} ms/img")
    except:
        pass

if __name__ == '__main__':
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    train_and_test_yolo_face()

🔄 Loading model base: yolo26n.pt...
🚀 Mulai Training YOLO26 Face ...
New https://pypi.org/project/ultralytics/8.4.8 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.1  Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=widerface.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26n.pt, momentum=0.937, m

c:\Users\mrcah\.conda\envs\tensorrt\lib\site-packages\torch\onnx\symbolic_opset9.py:5385: UserWarning: Exporting aten::index operator of advanced indexing in opset 19 is achieved by combination of multiple ONNX operators, including Reshape, Transpose, Concat, and Gather. If indices include negative values, the exported graph will produce incorrect results.
  warnings.warn(


ONNX: slimming with onnxslim 0.1.82...
ONNX: export success  5.8s, saved as 'yolo26n-face.onnx' (10.3 MB)

Export complete (6.4s)
Results saved to D:\1. Kuliah\Semester 7\test_code\YOLO26
Predict:         yolo predict task=detect model=yolo26n-face.onnx imgsz=640  
Validate:        yolo val task=detect model=yolo26n-face.onnx imgsz=640 data=widerface.yaml  
Visualize:       https://netron.app

📊 Melakukan Benchmark Akhir...
Ultralytics 8.4.1  Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
YOLO26n summary (fused): 122 layers, 2,375,031 parameters, 0 gradients, 5.2 GFLOPs
val: Fast image access  (ping: 0.20.0 ms, read: 12.54.0 MB/s, size: 129.7 KB)
val: Scanning D:\1. Kuliah\Semester 7\test_code\YOLO26\datasets\test\labels\0--Parade... 0 images, 16097 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 16097/16097 562.3it/s 28.6s<0.0s
WARNING val: No labels found in D:\1. Kuliah\Semester 7\test_code\YOLO26\datasets\test\labels\0--Parade.cache. See https://docs.ultralyti

c:\Users\mrcah\.conda\envs\tensorrt\lib\site-packages\ultralytics\utils\metrics.py:655: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
c:\Users\mrcah\.conda\envs\tensorrt\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\mrcah\.conda\envs\tensorrt\lib\site-packages\ultralytics\utils\metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
c:\Users\mrcah\.conda\envs\tensorrt\lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
c:\Users\mrcah\.conda\envs\tensorrt\lib\site-packages\ultralytics\utils\metrics.py:701: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
c:\Users\mrcah\.conda\envs\tensorrt\lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered 

                   all      16097          0          0          0          0          0
WARNING no labels found in detect set, cannot compute metrics without labels
Speed: 1.1ms preprocess, 5.0ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to D:\1. Kuliah\Semester 7\test_code\YOLO26\runs\detect\val

🏆 HASIL AKHIR (yolo26n-face.pt):
   🎯 mAP@50    : 0.00%
   🎯 mAP@50-95 : 0.00%
   ⚡ Speed     : 4.98 ms/img
